## 多输入多输出通道
- 每个输出通道可以识别特定模式， 例如有的通道找边缘，有的通道找颜色这种
- 一个输出通道对应一个三维的卷积核，所以多输出通道的卷积核是四维的权重
- 输入通道核识别并组合输入中的模式
- 刚开始学习很局部的特征，然后随着网络的加深，会把这些局部特征一步一步组合起来，组成一个整体的东西
### 1*1的卷积层
- **不识别空间模式，只是融合通道**融合不同通道的信息，并且可以改变通道数

- <img src='../images/21_image_1.png' height='200'>
- 相当于在通道维度做全连接
- 相当于输入形状为$n_hn_w \times c_i$权重为$c_o \times c_i$的全连接层，相当于在像素点上做一个全连接层的维度转换
- <img src='../images/21image2.png' width='500'>

- 多通道互相关运算

In [13]:
import torch
from torch import nn
from d2l import torch as d2l

# 对于多通道输入时单个输出通道的函数
def corr2d_multi_in(X, K):
    return sum([d2l.corr2d(x, k) for x, k in zip(X, K)])

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])
print(X[0].shape, X.shape, K.shape)
corr2d_multi_in(X, K)

torch.Size([3, 3]) torch.Size([2, 3, 3]) torch.Size([2, 2, 2])


tensor([[ 56.,  72.],
        [104., 120.]])

In [14]:
def corr2d_multi_out(X, K):
    """计算多通道输出的输出函数"""
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

# 理解一下torch.stack
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [15]:
corr2d_multi_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

- 1 * 1的卷积核实际上等价于是一个全连接层，因为他并没有涉及到对于图像尺寸的改变，只涉及到了通道数的改变，类似于像素的维度转换，例如三通道转换为2通道就类似于将每个像素点从3维转换成了2维

In [34]:
# 这个是按照自己的理解写的，与书上的版本有出入，但是思想上感觉差不多
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape(c_i, h * w)
    K = K.reshape(c_o, c_i)
    Y = torch.matmul(X.T, K.T)
    return Y.T.reshape(c_o, h, w)

# def corr2d_multi_in_out_1x1(X, K):
#     c_i, h, w = X.shape
#     c_o = K.shape[0]
#     X = X.reshape((c_i, h * w))
#     K = K.reshape((c_o, c_i))
#     # 全连接层中的矩阵乘法
#     Y = torch.matmul(K, X)
#     return Y.reshape((c_o, h, w))

X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_out(X, K)
Y2 = corr2d_multi_in_out_1x1(X, K)

assert float(torch.abs(Y1 - Y2).sum()) < 1e-6, Y1.shape

- 通道之间不共享参数，共享参数是对于同一个通道平移